# ****DISCLAIMER****

This dataset was acquired from a 3rd party source (Kaggle) and may contain inaccuracies or incomplete information.

# **Step 1**

Using Pandas, we will read in the rawdata CSV file. After reading it in, we will filter the data to include only the desired columns, ensuring we only grab the QB position and the years 2015-2024. Next, we will create new statistics to add to our cleaned-up dataset, which will then be exported as a CSV file into the cleandata folder.

In [2]:
import pandas as pd
#Using relative path read the raw nfl data
nfl_data_set = pd.read_csv("../rawdata/yearly_player_stats_offense.csv")

In [3]:
#Sort the data to only acquire columns needed for this project
#maybe look into adding playoffs
qb_data_needed = ["player_id", "player_name", "position", "season", "season_type", "team", "shotgun",
                  "no_huddle", "qb_dropback", "qb_scramble", "pass_attempts", "complete_pass",
                  "incomplete_pass", "passing_yards", "yards_after_catch", "rush_attempts",
                  "rushing_yards", "first_down_pass", "first_down_rush", "third_down_converted",
                  "third_down_failed", "fourth_down_converted", "fourth_down_failed", "rush_touchdown",
                  "pass_touchdown", "interception", "fumble", "fumble_lost", "passing_air_yards",
                  "pass_attempts_redzone", "complete_pass_redzone", "pass_touchdown_redzone", "pass_attempts_gtg", "complete_pass_gtg",
                  "pass_touchdown_gtg","rush_attempts_gtg", "rush_touchdown_gtg","offense_snaps", "team_offense_snaps", "total_tds", "touches",
                  "total_yards", "offense_pct", "games_played_season" ]

qb_data = nfl_data_set[qb_data_needed]

In [4]:
#Filter to only look at QB stats from 2015 to 2024
qb_data = qb_data[(qb_data["position"] == "QB")
            & qb_data["season"].between(2015,2024)]

In [5]:
qb_data.fillna(0, inplace=True)

In [6]:
#Create extra statistics for each qbs
qb_data["completion_pct"] = round((qb_data["complete_pass"] / qb_data["pass_attempts"]) * 100, 2)
qb_data["td_to_int_ratio"] = round(qb_data["pass_touchdown"] / qb_data["interception"], 2)
qb_data["red_zone_comp_pct"] = round(qb_data["complete_pass_redzone"] / qb_data["pass_attempts_redzone"] * 100, 2)
qb_data["total_turnovers"] =  qb_data["interception"] + qb_data["fumble_lost"]
qb_data["shotgun_pct"] = round(qb_data["shotgun"] / qb_data["offense_snaps"] *100, 2)
qb_data["dropback_pct"] = round(qb_data["qb_dropback"] / qb_data["offense_snaps"] *100, 2)
qb_data["yards_per_att"] = round(qb_data["passing_yards"] / qb_data["pass_attempts"], 2)
qb_data["td_per_att"] = round(qb_data["pass_touchdown"] / qb_data["pass_attempts"], 2)
qb_data["air_yds_att"] = round(qb_data["passing_air_yards"] / qb_data["pass_attempts"], 2)
qb_data["yds_per_rush"] = round(qb_data["rushing_yards"] / qb_data["rush_attempts"], 2)
qb_data["offense_pct"] = round(qb_data["offense_pct"],2)


In [7]:
#QBR Formula
a = ((qb_data["complete_pass"] / qb_data["pass_attempts"]) - 0.3) * 5
b = ((qb_data["passing_yards"] / qb_data["pass_attempts"]) - 3) * 0.25
c = (qb_data["pass_touchdown"] / qb_data["pass_attempts"]) * 20
d = 2.375 - ((qb_data["interception"] / qb_data["pass_attempts"]) * 25)

# Apply NFL bounds (0 to 2.375)
a = a.clip(0, 2.375)
b = b.clip(0, 2.375)
c = c.clip(0, 2.375)
d = d.clip(0, 2.375)

# Final Passer Rating
qb_data["passer_rating"] = round(((a + b + c + d) / 6) * 100,2)

In [8]:
#Export the cleaned up dataset into an Excel file for dashboard creation
qb_data.to_csv("../cleandata/cleaned_qb_stats.csv", index=False)

In [9]:
#Confirm the data types before moving to postgresql
qb_data.dtypes

player_id                  object
player_name                object
position                   object
season                      int64
season_type                object
team                       object
shotgun                     int64
no_huddle                   int64
qb_dropback                 int64
qb_scramble                 int64
pass_attempts               int64
complete_pass               int64
incomplete_pass             int64
passing_yards               int64
yards_after_catch           int64
rush_attempts               int64
rushing_yards               int64
first_down_pass             int64
first_down_rush             int64
third_down_converted        int64
third_down_failed           int64
fourth_down_converted       int64
fourth_down_failed          int64
rush_touchdown              int64
pass_touchdown              int64
interception                int64
fumble                      int64
fumble_lost                 int64
passing_air_yards           int64
pass_attempts_

## **STEP 2**

After exporting the cleaned-up data into a CSV file, you will move to a SQL platform, in this case PostgreSQL. First, create a database named "qbstats" (if you name it something else, please update dbname accordingly). Then, use the .sql files found in the sql folder to create the tables and views in that order.

After completing that, return to Python to export the views into a single Excel workbook, with each view as a separate worksheet. This workbook will then be used for the analysis portion of the project.

Please make sure your dbname, username, password, host, and port matches your systems.

In [10]:
#Connects to my postgresql server to create an Excel file
import psycopg2

conn = psycopg2.connect(
    dbname = "qbstats",
    user = "postgres",
    password = "123",
    host = "localhost",
    port = "5432",
)

In [11]:
#List of views created in my qb_data_views.sql file
views = [
    "top_30_pass_yards",
    "avg_yards_season",
    "top_30_passer_rating",
    "avg_yds_thrown_downfield",
    "top_30_total_yards",
    "avg_qb_rush_yards_season",
    "best_td_int_per_season",
    "top_playoff_performers"
]

In [12]:
#Name and location of the Excel Workbook
excel_ready_data = "../excel_ready_data/qb_data_created.xlsx"

In [14]:
#Might need to install openyxl for this code to work
with pd.ExcelWriter(excel_ready_data, engine="openpyxl") as writer:
    for view in views:
        df = pd.read_sql(f"SELECT * FROM {view};", conn)
        df.to_excel(writer, sheet_name=view, index=False)

C:\Users\Mike\AppData\Local\Temp\ipykernel_30160\3164809761.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {view};", conn)
